#4.5.3 다항분포 나이브 베이즈를 활용한 영화 리뷰 분류

※영화 리뷰,같은 단어가 여러 번 나오는지 확인->다항분포 나이브 베이즈

##실습 관련 라이브러리 임포트

In [2]:
import pandas as pd

# 다항분포 나이브 베이즈를 위한 라이브러리를 임포트합니다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# 모델의 정확도 평가를 위해 임포트합니다
from sklearn.metrics import accuracy_score

##데이터 획득

In [3]:
review_list = [
            {'movie_review': 'this is great great movie. I will watch again', 'type': 'positive'},
            {'movie_review': 'I like this movie', 'type': 'positive'},
            {'movie_review': 'amazing movie in this year', 'type': 'positive'},
            {'movie_review': 'cool my boyfriend also said the movie is cool', 'type': 'positive'},
            {'movie_review': 'awesome of the awesome movie ever', 'type': 'positive'},
            {'movie_review': 'shame I wasted money and time', 'type': 'negative'},
            {'movie_review': 'regret on this move. I will never never what movie from this director', 'type': 'negative'},
            {'movie_review': 'I do not like this movie', 'type': 'negative'},
            {'movie_review': 'I do not like actors in this movie', 'type': 'negative'},
            {'movie_review': 'boring boring sleeping movie', 'type': 'negative'}]
df = pd.DataFrame(review_list)
df

,movie_review,type
0,this is great great movie. I will watch again,positive
1,I like this movie,positive
2,amazing movie in this year,positive
3,cool my boyfriend also said the movie is cool,positive
4,awesome of the awesome movie ever,positive
5,shame I wasted money and time,negative
6,regret on this move. I will never never what m...,negative
7,I do not like this movie,negative
8,I do not like actors in this movie,negative
9,boring boring sleeping movie,negative


##데이터 다듬기

In [4]:
# 사이킷런의 다항분포 나이브 베이즈 분류기는 숫자만 다루기 때문에
#먼저 출력을 숫자로 변환
df['label'] = df['type'].map({"positive":1,"negative":0})
df

,movie_review,type,label
0,this is great great movie. I will watch again,positive,1
1,I like this movie,positive,1
2,amazing movie in this year,positive,1
3,cool my boyfriend also said the movie is cool,positive,1
4,awesome of the awesome movie ever,positive,1
5,shame I wasted money and time,negative,0
6,regret on this move. I will never never what m...,negative,0
7,I do not like this movie,negative,0
8,I do not like actors in this movie,negative,0
9,boring boring sleeping movie,negative,0


In [5]:
# 학습에 사용될 데이터와 분류값 나누기
df_x = df["movie_review"]
df_y = df["label"]

In [6]:
# 다항분포 나이브 베이즈 분류기의 입력 데이터(영화 리뷰)는 고정된 크기의 벡터이여야함.
# 입력 데이터를 모두 고정 길이로 만들기
cv = CountVectorizer() # ★★binary = True 가 없기 때문에 출현 횟수로 저장★★
x_traincv = cv.fit_transform(df_x)

# 전체 고정 벡터 인덱스의 단어 출력 (총 37개,알파벳 순)출력. (한 글자 단어는 제외됨. 예.I)
cv.get_feature_names()

['actors',
 'again',
 'also',
 'amazing',
 'and',
 'awesome',
 'boring',
 'boyfriend',
 'cool',
 'director',
 'do',
 'ever',
 'from',
 'great',
 'in',
 'is',
 'like',
 'money',
 'move',
 'movie',
 'my',
 'never',
 'not',
 'of',
 'on',
 'regret',
 'said',
 'shame',
 'sleeping',
 'the',
 'this',
 'time',
 'wasted',
 'watch',
 'what',
 'will',
 'year']

In [7]:
# 영화 감상평들의 인코딩 벡터 확인
encoded_input=x_traincv.toarray()
encoded_input

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 2,
        0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0

In [8]:
# 첫 번째 영화 감상평의 고정된 벡터에 포함된 단어 확인
cv.inverse_transform(encoded_input[0]) # 한 글자 단어는 포함 안 됨. (예.I)

[array(['again', 'great', 'is', 'movie', 'this', 'watch', 'will'],
       dtype='<U9')]

##Multinomial(다항분포) Naive Bayes 모델 학습

In [9]:
# 학습 데이터로 다항분포 분류기 학습
# 사이킷런의 다항분포 나이브 베이즈는 스무딩을 기본 지원함
mnb = MultinomialNB()
mnb.fit(x_traincv,df_y) # 다항분포 분류기 학습 실행

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

##테스트 데이터 다듬기

In [10]:
test_feedback_list = [
                {'movie_review': 'great great great movie ever', 'type': 'positive'},
                {'movie_review': 'I like this amazing movie', 'type': 'positive'},
                {'movie_review': 'my boyfriend said great movie ever', 'type': 'positive'},
                {'movie_review': 'cool cool cool', 'type': 'positive'},
                {'movie_review': 'awesome boyfriend said cool movie ever', 'type': 'positive'},
                {'movie_review': 'shame shame shame', 'type': 'negative'},
                {'movie_review': 'awesome director shame movie boring movie', 'type': 'negative'},
                {'movie_review': 'do not like this movie', 'type': 'negative'},
                {'movie_review': 'I do not like this boring movie', 'type': 'negative'},
                {'movie_review': 'aweful terrible boring movie', 'type': 'negative'}]
test_df = pd.DataFrame(test_feedback_list)
test_df['label'] = test_df['type'].map({"positive":1,"negative":0})
test_x=test_df["movie_review"]
test_y=test_df["label"]
x_testcv=cv.transform(test_x) # 고정 크기 벡터로 변환
# cv.fit_transform이 아님. 상단 고정 길이 벡터 작성 시 이미 fit(단어 학습)을 했기 때문

##테스트 진행

In [11]:
# 테스트 데이터에 대해 예측 실행
predictions = mnb.predict(x_testcv)
predictions

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [12]:
# 모델 예측 정확도(accuracy) 출력
accuracy = accuracy_score(test_y,predictions)
if accuracy ==1:
    print("accuracy : 100%")
else:
    print("accuracy : %.2f" %(accuracy_score(test_y,predictions)*100)+"%")

accuracy : 100%
